In [ ]:
from scipy.integrate import simps,trapz
import quadpy
import numpy as np
import torchquad
import torch
from time import perf_counter
from tqdm import tqdm

# Allow changing module on the fly
%load_ext autoreload
%autoreload 2
# Enable GPU usage
torchquad.enable_cuda()
# Set precision
np.set_printoptions(precision=10)
torch.set_printoptions(precision=10)
#Define how many iterations should be done as warmup in benchmarks
WARMUP_CALLS = 3

### Some test functions

In [ ]:
e = 2.7182818284590452353602874 #hardcoded to avoid dependence on torch / numpy
def test_function_1d(x):
    """ As a test function in 1D we use e^x
    """
    return e**x

def test_function_3d(x):
    """ As a test function in 3D we use the sum over e^(x) for each dim
    """
    return e**x[:,0] + e**x[:,1] + e**x[:,2]

def test_function_3d_quadpy(x):
    """ Same func as above but - for quadpy usage - with transposed dimensions
    """
    return e**x[0,:] + e**x[1,:] + e**x[2,:]

# Integration domains for the functions we use
test_domain_1d = [[0,1]]
test_domain_3d = [[0,5],[1,3],[-2,2]]

# Ground-truth solutions to compare to
ground_truth_1d = e - 1
ground_truth_3d = 1599.18758287212565283365376364

### Wrappers for all integrators

In [ ]:
def scipy_trapezoid_1d(fn,dim,N,domain):
    """Wrapper for scipy trapezoid method, creates grid as well
    """
    eval_points = torchquad.IntegrationGrid(N,domain).points.cpu().numpy().transpose()
    y = fn(eval_points)
    return trapz(y,eval_points)[0]

def scipy_simpsons_1d(fn,dim,N,domain):
    """Wrapper for scipy simpsons method, creates grid as well
    """
    eval_points = torchquad.IntegrationGrid(N,domain).points.cpu().numpy().transpose()
    y = fn(eval_points)
    return simps(y,eval_points)[0]


def _quadpy_helper(N,domain):
    """For quadpy we need to give it the border points of each cubic integration subdomain, this functions does that
    """
    
    #Create the integration points
    grid = torchquad.IntegrationGrid(N,domain)
    _N = grid._N #get points per dimension
    eval_points = grid.points.cpu().numpy() #get integration points

    # Discard all points that are unneeded to specify cubes on the grid (3 per cube needed instead of 8 available)
    cube_lower = eval_points[:-_N**2] #create two arrays with offset
    cube_higher = eval_points[_N**2:]
    z_mask = cube_lower[:,2] != domain[2][1] # for lower z discard points at high end of boundary
    cube_lower = cube_lower[z_mask]
    z_mask = cube_higher[:,2] != domain[2][0] # for higher z discard points at low end of boundary
    cube_higher = cube_higher[z_mask]
    y_mask = cube_lower[:,1] != domain[1][1] # as above but for y 
    cube_lower = cube_lower[y_mask]
    y_mask = cube_higher[:,1] != domain[1][0] # as above but for y 
    cube_higher = cube_higher[y_mask]
    
    x_dims = np.vstack([cube_lower[:,0],cube_higher[:,0]]).T # X coordinates
    y_dims = np.vstack([cube_lower[:,1],cube_higher[:,1]]).T # Y coordinates
    z_dims = np.vstack([cube_lower[:,2],cube_higher[:,2]]).T # Z coordinates
    
    cubes = []
    
    #Collect all cubes
    for x,y,z in zip(x_dims,y_dims,z_dims):
        cubes.append(quadpy.c3.cube_points(x,y,z))
    cubes = np.stack(cubes,axis=-2)
    
    return cubes

def quadpy_trapezoid_3d(fn,dim,N,domain):
    """3D trapezoid using quadpy, note that it will perform 8*N evals
    """
    cubes = _quadpy_helper(N,domain)
#     plot_cubes(cubes)
    scheme = quadpy.c3.product(quadpy.c1.newton_cotes_closed(1))
    val = scheme.integrate(fn,cubes)
    return 0.5*sum(val) #this 0.5 is very odd because it implies that we integrate each domain twice, however we don't?

def quadpy_simpson_3d(fn,dim,N,domain):
    """3D simpsons using quadpy, note that it will perform 27*N evals
    """
    cubes = _quadpy_helper(N,domain)
#     plot_cubes(cubes)
    scheme = quadpy.c3.product(quadpy.c1.newton_cotes_closed(2))
    val = scheme.integrate(fn,cubes)
    return 0.5*sum(val) #this 0.5 is very odd because it implies that we integrate each domain twice, however we don't?
    
def torchquad_trapezoid(fn,dim,N,domain):
    """Wrapper for scipy simpsons method, creates grid as well
    """
    tp = torchquad.Trapezoid()
    return tp.integrate(fn,dim,N,domain).item()
    
def torchquad_simpsons(fn,dim,N,domain):
    """Wrapper for scipy simpsons method, creates grid as well
    """
    sp = torchquad.Simpson()
    return sp.integrate(fn,dim,N,domain).item()

### Benchmarking utils

In [ ]:
def evaluate(integrator,fn,dim,N,domain):
    """Evaluates given integrator and returns result and runtime
    """
    start = perf_counter()
    result = integrator(fn,dim,N,domain)
    time = perf_counter() - start
    return result,time

def benchmark(integrator,fn,dim,N,domain, iterations=10):
    """ Evaluates given Integrator once as warmup, then #iterations times
    """
    results,times = [],[]
    #Do three calls as warmup (to allow CPU / GPU to use caching and increase frequency)
    for i in range(WARMUP_CALLS): 
        evaluate(integrator,fn,dim,N,domain)
    #Do iteration benchmark calls
    for it in range(iterations):
        result,time = evaluate(integrator,fn,dim,N,domain)
        results.append(result)
        times.append(time)
    return results,times

### Compute all runtimes and results for 1D example

In [ ]:
# Select precision: Default it is float32 , can be set to double with this
torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
methods = [scipy_trapezoid_1d,scipy_simpsons_1d,torchquad_trapezoid,torchquad_simpsons]
values, times, names, steps_per_method = [],[],[],[]
steps = 2**np.linspace(1,20,20) + 1
for method in methods:
    cur_values,cur_time = [],[]
    for step in tqdm(steps):
        vals,time_measuresments = benchmark(method,test_function_1d,1,int(step),test_domain_1d)
        cur_values.append(vals[0]) # All methods are deterministic, so we just take one value
        cur_time.append(np.mean(time_measuresments)) # For time take the mean
    names.append(method.__name__)
    steps_per_method.append(steps)
    values.append(cur_values)
    times.append(cur_time)

In [ ]:
torchquad.plot_convergence(steps_per_method,values,ground_truth_1d,names)

In [ ]:
torchquad.plot_runtime(steps_per_method,times,names)

### Compute runtimes and results for the 3D example

In [ ]:
methods = [quadpy_trapezoid_3d,quadpy_simpson_3d,torchquad_trapezoid,torchquad_simpsons]
values, times, names, steps_per_method = [],[],[],[]
steps = np.asarray([3,7,11,15,19])**3
for method in methods:
    cur_values,cur_time = [],[]
    for step in tqdm(steps):
        if method in [quadpy_trapezoid_3d,quadpy_simpson_3d]:
            vals,time_measuresments = benchmark(method,test_function_3d_quadpy,3,int(step),test_domain_3d)
        else:
            vals,time_measuresments = benchmark(method,test_function_3d,3,int(step),test_domain_3d)
        cur_values.append(vals[0]) # All methods are deterministic, so we just take one value
        cur_time.append(np.mean(time_measuresments)) # For time take the mean
    names.append(method.__name__)
    #correct for extra evals that quadpy performs
    if method is quadpy_trapezoid_3d: 
        steps = steps * 8
    if method is quadpy_simpson_3d:
        steps = steps * 27
    steps_per_method.append(steps)
    values.append(cur_values)
    times.append(cur_time)

In [ ]:
torchquad.plot_convergence(steps_per_method,values,ground_truth_3d,names)

In [ ]:
torchquad.plot_runtime(steps_per_method,times,names)

In [ ]:
import matplotlib.pyplot as plt
def plot_cubes(cubes):
    points = cubes.reshape([-1,3])
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:, 0],
               points[:, 1],
               points[:, 2])